In [1]:
from conflux_web3 import Web3 as ConfluxWeb3
from web3 import Web3
from web3.middleware.signing import construct_sign_and_send_raw_middleware

import os, json, dotenv
from typing import Any, TypedDict

dotenv.load_dotenv()


def get_sk() -> str:
    return os.environ["SECRET_KEY"]


def get_e_web3() -> Web3:
    return Web3(Web3.HTTPProvider(os.environ["EVM_URL"]))


def get_c_web3() -> ConfluxWeb3:
    return ConfluxWeb3(ConfluxWeb3.HTTPProvider(os.environ["CORE_URL"]))


class Metadata(TypedDict):
    abi: Any
    bytecode: Any


def get_metadata(name: str) -> Metadata:
    with open(f"build/contracts/{name}.json") as f:
        d = json.load(f)
        return {
            "abi": d["abi"],
            "bytecode": d["bytecode"]
        }

c_web3 = get_c_web3()
e_web3 = get_e_web3()
c_web3.cfx.default_account = c_web3.account.from_key(get_sk())
e_account = e_web3.eth.account.from_key(get_sk())
e_web3.middleware_onion.add(construct_sign_and_send_raw_middleware(e_account))
e_web3.eth.default_account = e_account.address

csc_address = c_web3.cfx.contract(name="CrossSpaceCall").address
sponsor_whitelist_control = c_web3.cfx.contract(name="SponsorWhitelistControl", with_deployment_info=True)
name = os.environ["CORE_CONTRACT_NAME"]
symbol = os.environ["CORE_CONTRACT_SYMBOL"]
default_oracle_life = int(os.environ["DEFAULT_ORACLE_LIFE"])

DualSpaceNFTCore = c_web3.cfx.contract(**get_metadata("DualSpaceNFTCore"))
DualSpaceNFTEvm = e_web3.eth.contract(**get_metadata("DualSpaceNFTEvm"))
DeploymentProxyCore = c_web3.cfx.contract(**get_metadata("DeploymentProxy"))
DeploymentProxyEvm = e_web3.eth.contract(**get_metadata("DeploymentProxy"))
espace_chain_id = e_web3.eth.chain_id

In [2]:
print(f"deploying...")
core_impl_addr = DualSpaceNFTCore.constructor().transact().executed()["contractCreated"]
print(f"core contract impl deployed at {core_impl_addr}")
core_contract = DualSpaceNFTCore(DeploymentProxyCore.constructor(core_impl_addr, "0x").transact().executed()["contractCreated"])
print(f"core contract proxy deployed at {core_contract.address}")

evm_impl_addr = e_web3.eth.wait_for_transaction_receipt(DualSpaceNFTEvm.constructor().transact({
    "gasPrice": 2 * 10 **10
}))["contractAddress"]
print(f"evm contract impl deployed at {evm_impl_addr}")
evm_contract = DualSpaceNFTEvm(e_web3.eth.wait_for_transaction_receipt(DeploymentProxyEvm.constructor(evm_impl_addr, "0x").transact({
    "gasPrice": 2 * 10 **10
}))["contractAddress"])
print(f"evm contract proxy deployed at {evm_contract.address}")


deploying...
core contract impl deployed at cfx:acff58p0z6mtcy6vt0crjgtsvgtvkh92568gc1rwzn
core contract proxy deployed at CFX:TYPE.CONTRACT:ACD97PBHXM11CEGRZ3ANUUVCUJ9J0NH956A108F5CC
evm contract impl deployed at 0x634757eFE5DD3D27ecf38480c6F2Eac6752E90DB


ValueError: {'code': -32000, 'message': 'allowed qps exceeded: Too many requests (exceeds 50), try again after 56ms'}

In [4]:
evm_contract = DualSpaceNFTEvm("0x500371e7Ec0B5BCA911A11964300E694bb0FEc9D")

In [5]:

print("initializing contracts...")
mapped_address = c_web3.address.calculate_mapped_evm_space_address(core_contract.address)
evm_contract.functions.initialize(
    name, symbol, mapped_address
).transact({
    "gasPrice": 2 * 10 **10
})
core_contract.functions.initialize(
    name, symbol, evm_contract.address, csc_address, espace_chain_id, default_oracle_life
).transact().executed()

initializing contracts...


AttributeDict({'transactionHash': HexBytes('0x02c71566ef37021a0272013ba8e4833b90291b5ae738524e4cd746fc96420516'),
 'index': 0,
 'blockHash': HexBytes('0x03130a25dff42b5cec7e34896aa3940670344f2bfda8099dbfccc3593be9bda2'),
 'epochNumber': 78330471,
 'from': 'cfx:aanthezva9vuyrpsmb15sf9xt97f8vduky1k1vwka5',
 'to': 'cfx:acd97pbhxm11cegrz3anuuvcuj9j0nh956a108f5cc',
 'gasUsed': 118764,
 'gasFee': 118764000000000 Drip,
 'contractCreated': None,
 'logs': [AttributeDict({'address': 'cfx:acd97pbhxm11cegrz3anuuvcuj9j0nh956a108f5cc',
   'topics': [HexBytes('0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0'),
    HexBytes('0x0000000000000000000000000000000000000000000000000000000000000000'),
    HexBytes('0x00000000000000000000000016f392b107e30a358e506fb717f37ffa5f44704d')],
   'data': HexBytes('0x')}),
  AttributeDict({'address': 'cfx:acd97pbhxm11cegrz3anuuvcuj9j0nh956a108f5cc',
   'topics': [HexBytes('0x7f26b83ff96e1f2b6a682f133852f6798a09c465da95921460cefb3847402498')],
   'dat

In [6]:
print("setting sponsor...")
sponsor_whitelist_control.functions.addPrivilegeByAdmin(core_contract.address, [c_web3.address.zero_address()]).transact().executed()
sponsor_whitelist_control.functions.setSponsorForCollateral(core_contract.address).transact({
    "value": 2 * 10**19
}).executed()
sponsor_whitelist_control.functions.setSponsorForGas(core_contract.address, 10**16).transact({
    "value": 10**19
}).executed()


setting sponsor...


AttributeDict({'transactionHash': HexBytes('0x86c3fb0d0fef26e652f408a84fbaeffea26f45462679a1ad159eb0b41b0c9967'),
 'index': 0,
 'blockHash': HexBytes('0xde9213a4528c7f3be60e0d40a1ffaae25462e84ce611ffde3ef1945c0847c8c1'),
 'epochNumber': 78330590,
 'from': 'cfx:aanthezva9vuyrpsmb15sf9xt97f8vduky1k1vwka5',
 'to': 'cfx:aaejuaaaaaaaaaaaaaaaaaaaaaaaaaaaaegg2r16ar',
 'gasUsed': 33128,
 'gasFee': 33128000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 'sta

In [9]:
authorizer = c_web3.cfx.account.create()
signer = c_web3.cfx.account.create()
core_contract.functions.startBatch(20230401, signer.address, authorizer.address, 8).transact().executed()

AttributeDict({'transactionHash': HexBytes('0x649fd1658e1b8201ad96ec75e3b334f36f807e1ead39ffb4dd85ebeb77f6a23c'),
 'index': 0,
 'blockHash': HexBytes('0xe7af5ce423332af6ee2e61b0144d1952de7fe515381cc818bcc3e3bd54a71f6b'),
 'epochNumber': 78331859,
 'from': 'cfx:aanthezva9vuyrpsmb15sf9xt97f8vduky1k1vwka5',
 'to': 'cfx:acd97pbhxm11cegrz3anuuvcuj9j0nh956a108f5cc',
 'gasUsed': 178494,
 'gasFee': 849799000000000 Drip,
 'contractCreated': None,
 'logs': [AttributeDict({'address': 'cfx:aaejuaaaaaaaaaaaaaaaaaaaaaaaaaaaa2sn102vjv',
   'topics': [HexBytes('0x124d1efd3ac32fa6aadb7a06e066c113471e0411fb56a5079cedbc3cdf7796e4'),
    HexBytes('0x94195a79a8ce9ddd108ae2985e9f3cf99ced7842000000000000000000000000'),
    HexBytes('0x500371e7ec0b5bca911a11964300e694bb0fec9d000000000000000000000000')],
   'data': HexBytes('0x000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [18]:
c_web3.wallet.add_account(authorizer)
core_contract.functions.batchAuthorizeMintPermission(20230401,[
    "crowdin-w33d", "crowdin-cyclee", "crowdin-jasonyou184", "crowdin-louisl.eth", "crowdin-effectchen", "crowdin-cike", "github-Euroanchor", "github-atharvadeosthale", "github-wanliqun", "github-Pana", "github-0xn1c0", "github-darwintree"
], [5] + [4] * 11).transact().executed()

AttributeDict({'transactionHash': HexBytes('0x31f54aebac8f03cc076b9bbe90c625fade36b2d354c68c9164b09bddb6881acc'),
 'index': 1,
 'blockHash': HexBytes('0x41d609dd8132776917767be3cfc08993096dc2066579a75a6bd929a8611afe8c'),
 'epochNumber': 78332758,
 'from': 'cfx:aanthezva9vuyrpsmb15sf9xt97f8vduky1k1vwka5',
 'to': 'cfx:acd97pbhxm11cegrz3anuuvcuj9j0nh956a108f5cc',
 'gasUsed': 126212,
 'gasFee': 126212000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'),
 's

In [17]:
authorizer